In [1]:
from DTC.dtc_executor import DTCExecutor
from DTC.route_skeleton import RouteSkeleton
import json
from math import ceil

In [2]:
import pickle

#a = DTCExecutor()
#pc = a.create_point_cloud_with_n_points(17000000, city=True)



In [3]:
#with open('./pc.pickle', 'wb') as handle:
#    pickle.dump(pc, file=handle)

In [4]:
pc = None
with open('./pc.pickle', 'rb') as handle:
    pc = pickle.load(handle)

In [5]:
from DTC.gridsystem import GridSystem
from DTC.construct_safe_area import SafeArea
gs = GridSystem(pc)
route_skeleton = set()
with open("./AllcityRSK.json", "r") as file:
    data = json.load(file)
    for elem in data:
        clean_elem = elem.strip('()').split(',')

        set_elem = tuple(float(num.strip()) for num in clean_elem)

        route_skeleton.add(set_elem)

        print(set_elem)
        print(type(set_elem))
converted_data = {}
with open("./AllcitySA.json", "r") as file:
    data = json.load(file)
    for key, value in data.items():
        
        
        clean_key = key.strip('()').split(',')
        tuple_key = tuple(float(num.strip()) for num in clean_key)

        converted_data[tuple_key] = SafeArea(tuple_key, radius=value[0], cardinality=value[1], confidence_change=1, normalisation_factor=0.5, cardinality_squish=0.15, max_confidence_change=0.15)

        print(tuple_key, value)

gs.safe_areas = converted_data
gs.route_skeleton = route_skeleton


(2482.2, 3730.16)
<class 'tuple'>
(4815.22, 4091.31)
<class 'tuple'>
(5149.34, 5688.64)
<class 'tuple'>
(2385.97, 811.35)
<class 'tuple'>
(5027.7, 2408.81)
<class 'tuple'>
(4037.25, 2970.85)
<class 'tuple'>
(2834.06, 2903.49)
<class 'tuple'>
(1727.88, 3212.61)
<class 'tuple'>
(3310.44, 2683.07)
<class 'tuple'>
(3868.15, 2576.36)
<class 'tuple'>
(3603.69, 3274.53)
<class 'tuple'>
(1392.4, 2520.29)
<class 'tuple'>
(2510.43, 2857.49)
<class 'tuple'>
(3231.23, 2503.24)
<class 'tuple'>
(4428.57, 5721.19)
<class 'tuple'>
(3897.02, 2406.06)
<class 'tuple'>
(3375.26, 4546.38)
<class 'tuple'>
(4814.5, 3859.36)
<class 'tuple'>
(4612.57, 4488.59)
<class 'tuple'>
(3632.35, 3814.42)
<class 'tuple'>
(2026.5, 2179.08)
<class 'tuple'>
(1913.4, 3005.27)
<class 'tuple'>
(3023.16, 2652.42)
<class 'tuple'>
(3598.69, 761.81)
<class 'tuple'>
(751.96, 3011.21)
<class 'tuple'>
(3640.83, 2916.04)
<class 'tuple'>
(2258.25, 4123.73)
<class 'tuple'>
(2480.48, 2782.35)
<class 'tuple'>
(4403.82, 3670.7)
<class 'tup

In [6]:
import copy
i = 0
for traj in gs.pc.trajectories:
    i+=1
#old_traj = copy.deepcopy(gs.pc.trajectories)
print(i)

302578


In [7]:
#extract r
import random
traj_for_test = random.sample(gs.pc.trajectories, 1467)



In [8]:
i = 0
for traj in traj_for_test:
    for point in traj.points:
        i += 1
print(i)

57750


In [9]:
# use 5% of 52489 - or 5 % of all the points in the collective pointcloud for all trajectories
from math import ceil
amount_to_shift = ceil(i * 0.05)
print(amount_to_shift, " and ", i)

2888  and  57750


***We use random.sample to get a random set of trajectories every time this runs***
- furthermore, we take 5% of the points and shift them between 1-200m randomly 
- this is done to test the cleaning of these

In [10]:
from DTC.distance_calculator import DistanceCalculator
list_of_points = []
for traj in traj_for_test:
    for point in traj.points:
        print(point.longitude)
        list_of_points.append(point)
list_of_points = copy.copy(list_of_points)

points_to_shift = random.sample(list_of_points, amount_to_shift)
print(len(points_to_shift))

116.4654
116.46541
116.46545
116.46547
116.46547
116.46542
116.46542
116.46542
116.46543
116.46518
116.4655
116.46539
116.46542
116.46541
116.46544
116.46522
116.46529
116.46534
116.46536
116.46538
116.4654
116.46541
116.46541
116.46541
116.46541
116.46541
116.46541
116.46542
116.46542
116.46542
116.46543
116.46542
116.46542
116.46544
116.46544
116.46543
116.46542
116.46541
116.4653
116.46533
116.46533
116.46534
116.46534
116.46534
116.46534
116.46534
116.46534
116.46535
116.46535
116.46535
116.46582
116.46549
116.46548
116.46527
116.46527
116.46528
116.46523
116.46526
116.46538
116.4658
116.46564
116.46549
116.46546
116.46546
116.46542
116.46543
116.46543
116.46543
116.46545
116.46556
116.46552
116.46549
116.46549
116.46549
116.46549
116.46548
116.46548
116.46548
116.46547
116.46547
116.46547
116.46548
116.46548
116.46548
116.46548
116.46548
116.46548
116.4655
116.46552
116.46561
116.46539
116.4654
116.46543
116.46549
116.46528
116.46543
116.46546
116.46544
116.46546
116.4656
116.4655

In [11]:
from DTC.distance_calculator import DistanceCalculator
from DTC.point import Point
from datetime import datetime
#Shifting the points twice seem to give me coordinate that does not work - this does not really make any sense

for point in points_to_shift:
    print(point.longitude)
    point_shifted = DistanceCalculator.shift_point_with_bearing(point, random.randint(1,200), random.randint(0,359))
    point.longitude = point_shifted[0]
    point.latitude = point_shifted[1]
    point.noise = True
    #point = DistanceCalculator.shift_point_with_bearing_and_alter_point(point, random.randint(1,200), DistanceCalculator.EAST, gs.initialization_point)

116.40109
116.40235
116.42269
116.32589
116.32877
116.3852
116.3528
116.35127
116.29728
116.31752
116.43351
116.35698
116.45567
116.33247
116.35284
116.38118
116.41639
116.4193
116.32925
116.41791
116.44125
116.4814
116.42112
116.40869
116.3486
116.45534
116.4062
116.28579
116.4761
116.3481
116.45574
116.34908
116.44802
116.42158
116.46345
116.47335
116.47933
116.38778
116.37098
116.42968
116.34507
116.44954
116.46518
116.36853
116.29771
116.31951
116.36043
116.25332
116.32847
116.38807
116.34156
116.36662
116.26906
116.44914
116.32285
116.37627
116.41176
116.3361
116.40905
116.40123
116.43342
116.36901
116.47942
116.34564
116.41607
116.44842
116.44752
116.43943
116.2446
116.42063
116.38792
116.38553
116.41052
116.41494
116.45486
116.45452
116.47324
116.41708
116.32554
116.44375
116.45372
116.32162
116.42127
116.35764
116.35898
116.3134
116.47469
116.34625
116.36802
116.4616
116.35021
116.41963
116.47529
116.44661
116.34185
116.41745
116.41008
116.41142
116.44275
116.40171
116.42368
11

In [12]:
for point in points_to_shift:
    print(point.longitude)

116.4015568591185
116.40238256721469
116.42406181611317
116.32552120285031
116.3288656019798
116.38424678769168
116.35254418723946
116.35046915498505
116.29804336937859
116.31891489105489
116.43338627924841
116.35542171550475
116.45698113559877
116.33376863460944
116.35328969097468
116.38079035461126
116.41631478613625
116.41956096209928
116.32927004951523
116.41794682280631
116.441263059438
116.4813702125776
116.42045242087701
116.41028850909255
116.34932285436749
116.45636082169189
116.40614512575159
116.28432463492369
116.47657657616043
116.3492654109266
116.45691762491138
116.34886455329774
116.44773584756837
116.42378729059442
116.46425564076884
116.4748637819989
116.47852339802151
116.38843654546669
116.37117490104535
116.42970245829339
116.34472121072405
116.45084445503791
116.46719542774696
116.3680340001056
116.29809043088767
116.3201372773073
116.35896835306119
116.25360944616163
116.3289496956019
116.38816688199302
116.34137320603105
116.36656651290068
116.26952504299595
116

In [13]:
from DTC import clean
clean_t = clean.CleanTraj(gs.safe_areas, gs.route_skeleton, gs.initialization_point)


In [14]:

cleaned_points = clean_t.clean(list_of_points)

- After Cleaning, make comparison with the old trajectory points

In [15]:
print(len(cleaned_points))
i = 0
for point in cleaned_points:
    if point == True:
        i += 1
print(i)
i = 0
for point in list_of_points:
    if point.noise == True:
        i += 1
print(i)

881
257
2888


**Test Setup**

- Using the setup from the paper DTC
- Using 1472 trajectories as in the paper in the first set of experiments
- 



**This is only with a MR with 2million points**

*first run*
- 926 cleaned
- 3107 to clean

*second run*
- 1140 cleaned
- 2564 to clean


**Running with 11 million points**

*first run*
- 739 cleaned
- 2573 to clean

*second run*
- 775 cleaned
- 2342 to clean

*third run*
- 850 cleaned
- 2394 to clean

*fourth run*
- 656 cleaned
- 2386 to clean

*fifth run*
- 1137 cleaned
- 3998 to clean

- VERY low amount of true positives - thats bad


In [16]:
average_cleaned = sum([739, 775, 850, 656]) / sum([2573, 2342, 2394, 2386])
print(average_cleaned)

0.3115007735946364
